In [6]:
import numpy as np
import pandas as pd
from nltk.corpus import brown
from tqdm.notebook import tqdm

In [7]:
nltk.download('brown')

[nltk_data] Downloading package brown to /Users/snjev310/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [8]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/snjev310/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [9]:
data = brown.tagged_sents(tagset = "universal")

In [11]:
len(data)

57340

In [12]:
type(data)

nltk.corpus.reader.util.ConcatenatedCorpusView

In [13]:
data = list(data)

In [160]:
#data

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import brown

nltk.download('brown')
nltk.download('universal_tagset')

data = brown.tagged_sents(tagset = "universal")
data = list(data)

[nltk_data] Downloading package brown to /Users/snjev310/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/snjev310/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [2]:
len(data)

57340

In [3]:
#Add tags for start of sentence ^ for ^
def preprocess(data):
    for s in data:
        s.insert(0,('^','*'))

In [4]:
preprocess(data)

In [5]:
data = data[0:100]
data[3]

[('^', '*'),
 ('``', '.'),
 ('Only', 'ADV'),
 ('a', 'DET'),
 ('relative', 'ADJ'),
 ('handful', 'NOUN'),
 ('of', 'ADP'),
 ('such', 'ADJ'),
 ('reports', 'NOUN'),
 ('was', 'VERB'),
 ('received', 'VERB'),
 ("''", '.'),
 (',', '.'),
 ('the', 'DET'),
 ('jury', 'NOUN'),
 ('said', 'VERB'),
 (',', '.'),
 ('``', '.'),
 ('considering', 'ADP'),
 ('the', 'DET'),
 ('widespread', 'ADJ'),
 ('interest', 'NOUN'),
 ('in', 'ADP'),
 ('the', 'DET'),
 ('election', 'NOUN'),
 (',', '.'),
 ('the', 'DET'),
 ('number', 'NOUN'),
 ('of', 'ADP'),
 ('voters', 'NOUN'),
 ('and', 'CONJ'),
 ('the', 'DET'),
 ('size', 'NOUN'),
 ('of', 'ADP'),
 ('this', 'DET'),
 ('city', 'NOUN'),
 ("''", '.'),
 ('.', '.')]

In [8]:
import random
def createFold(data,k):
    n = len(data)
    n_splits = n//k
    random.shuffle(data)
    folds = []
    for i in range(0,n,n_splits):
        j = min(i+n_splits,n)
        folds.append(data[i:j])
    return folds

In [9]:
data_f = createFold(data,5)

In [10]:
len(data_f[4])

20

In [11]:
def uniqueTags(data):
    tags = set()
    for s in data:
        for word,tag in s:
            tags.add(tag)
    return list(sorted(tags))

In [12]:
tags = uniqueTags(data)
n_tags = len(tags)

In [13]:
def vocab_(data):
    vocab = set()
    for s in data:
        for word,tag in s:
            vocab.add(word)
    return list(sorted(vocab))

In [79]:
vocab = vocab_(data)
n_vocab = len(vocab)


In [81]:
n_vocab

861

In [15]:
#HMM

In [20]:
from collections import defaultdict
def computeFrequency(data):
    transition_freq = defaultdict(int)
    emission_freq = defaultdict(int)
    tag_frequency = defaultdict(int)
    
    for s in data:
        previous_tag = '^'
        for word,tag in s:
            #print(word,tag)
            transition_freq[(tag,previous_tag)] += 1
            emission_freq[(word,tag)] += 1
            tag_frequency[tag] += 1
            previous_tag = tag
    return transition_freq,emission_freq,tag_frequency

In [21]:
tf,ef,tag_freq = computeFrequency(data)

In [159]:
#computeFrequency(data)

In [44]:
tag_freq

defaultdict(int,
            {'*': 100,
             'DET': 279,
             'NOUN': 721,
             'VERB': 384,
             'PRON': 48,
             '.': 259,
             'ADJ': 120,
             'ADP': 256,
             'ADV': 58,
             'PRT': 57,
             'NUM': 39,
             'CONJ': 47})

In [155]:
def buildMatrix(transition_freq,emission_freq,tag_freq,n_tags,n_vocab):
    transition_matrix = np.zeros((n_tags,n_tags))
    emission_matrix = np.zeros((n_tags,n_vocab))
    
    for i in range(n_tags):
        #transition matrix
        for j in range(n_tags):
            t_i = tags[i]
            t_j = tags[j]
            transition_matrix[i,j] = transition_freq[(t_i,t_j)]/tag_freq[t_i]
        #emission
        for j in range(n_vocab):
            t_i = tags[i]
            w_j = vocab[j]
            #print(w_j,t_i)
            emission_matrix[i,j] = (emission_freq[(w_j,t_i)])/tag_freq[t_i]
    #print(transition_matrix,emission_matrix)
    return transition_matrix, emission_matrix

In [156]:
tf,ef,tag_freq = computeFrequency(data)
tm,em = buildMatrix(tf,ef,tag_freq,n_tags,n_vocab)

In [158]:
em.shape

(12, 861)

In [122]:
tag_freq

defaultdict(int,
            {'*': 100,
             'DET': 279,
             'NOUN': 721,
             'VERB': 384,
             'PRON': 48,
             '.': 259,
             'ADJ': 120,
             'ADP': 256,
             'ADV': 58,
             'PRT': 57,
             'NUM': 39,
             'CONJ': 47})

In [161]:
# em= np.zeros((n_tags,n_vocab))
# for i in range(n_tags):
#     #transition matrix
#     for j in range(n_vocab):
#         t_i = tags[i]
#         w_j = vocab[j]
#         #print(t_i,w_j)
#         #print(ef[(t_i,w_j)])
#         em[i,j] = (ef[(w_j,t_i)])/tag_freq[t_i]
#     print(em)


In [87]:
w_j

'1'